In [2]:
!pip install flask

  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached blinker-1.8.2-py3-none-any.whl.metadata (1.6 kB)
Using cached flask-3.0.3-py3-none-any.whl (101 kB)
Using cached blinker-1.8.2-py3-none-any.whl (9.5 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)


In [3]:
from flask import Flask, Response, stream_with_context
import cv2
import requests
import numpy as np

app = Flask(__name__)

In [4]:
# URL to receive the video feed from Raspberry Pi
RPI_VIDEO_URL = "http://192.168.107.54:5000/video_feed"  # Replace with actual Raspberry Pi IP

In [5]:
def process_frame(frame):
    # Apply some processing (e.g., convert to grayscale)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    processed_frame = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    return processed_frame

In [6]:
def generate_processed_frames():
    stream = requests.get(RPI_VIDEO_URL, stream=True)
    byte_buffer = b''
    for chunk in stream.iter_content(chunk_size=1024):
        byte_buffer += chunk

        a = byte_buffer.find(b'\xff\xd8')  # Start of JPEG
        b = byte_buffer.find(b'\xff\xd9')  # End of JPEG

        if a != -1 and b != -1:
            jpg = byte_buffer[a:b+2]
            byte_buffer = byte_buffer[b+2:]

            # Decode frame and process
            frame = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
            processed_frame = process_frame(frame)

            # Send confirmation message back to Raspberry Pi after processing
            requests.post("http://192.168.107.54:5000/processing_done", json={"message": "Processing done for this frame"})

            # Encode the processed frame back to JPEG
            ret, buffer = cv2.imencode('.jpg', processed_frame)
            if not ret:
                continue

            processed_frame = buffer.tobytes()

            # Yield the processed frame as bytes
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + processed_frame + b'\r\n')

In [7]:
@app.route('/')
def index():
    # Automatically redirect to /processed_feed
    return Response("Redirecting to /processed_feed", status=302, headers={'Location': '/processed_feed'})

@app.route('/processed_feed')
def processed_feed():
    return Response(stream_with_context(generate_processed_frames()), mimetype='multipart/x-mixed-replace; boundary=frame')

In [8]:
app.run(host='0.0.0.0', port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.107.172:5001
Press CTRL+C to quit
192.168.107.172 - - [24/Oct/2024 23:56:21] "GET / HTTP/1.1" 302 -
192.168.107.172 - - [24/Oct/2024 23:56:24] "GET /processed_feed HTTP/1.1" 200 -
